# download-them-all
- used to be a browser plugin, but it broke, so i wrote a python version
- copy into a directory all the urls that match a pattern

# new tools used
- [urllib.parse](https://docs.python.org/3/library/urllib.parse.html)
- [named tuples](https://docs.python.org/3/glossary.html#term-named-tuple)
- functions [any and all](https://docs.python.org/3/library/functions.html#all)

In [6]:
import bs4
import lxml
import urllib.request
import urllib.parse
import os
import time
import re
from contextlib import contextmanager


In [7]:
@contextmanager
def withChdir(newdir):
    savedir = os.getcwd()
    os.chdir(newdir)
    try:
        yield
    finally:
        os.chdir(savedir)

print(os.getcwd())

with withChdir('/'):
    print( os.getcwd())
        
print(os.getcwd()) 
    

/Users/dbenson30/Desktop/python/lectures
/
/Users/dbenson30/Desktop/python/lectures


In [8]:
def dta(url, types=['pdf']):
    "return list of urls to fetch"
    # guarantee 'net' will be closed
    with urllib.request.urlopen(url) as net:
        soup = bs4.BeautifulSoup(net, 'lxml', from_encoding='utf-8')
    res = []
    alist = soup.findAll('a')
    for a in alist:
        href = a.get('href')
        # only take urls of right type
        # any is True if any element of the iterable is True
        if any([href.endswith(type) for type in types]):
            href = urllib.parse.urljoin(url, href)
            res.append(href)
    return res

# example page with with many downloads 
- [black holes](http://www.eftaylor.com/exploringblackholes/)

In [9]:
dta('http://www.eftaylor.com/exploringblackholes/')

['http://www.eftaylor.com/exploringblackholes/?path=&download=AAAAREADME180423v1.pdf',
 'http://www.eftaylor.com/exploringblackholes/AAAAREADME180423v1.pdf',
 'http://www.eftaylor.com/exploringblackholes/AAAAREADME180423v1.pdf',
 'http://www.eftaylor.com/exploringblackholes/?path=&download=AAAFrontCover171026.pdf',
 'http://www.eftaylor.com/exploringblackholes/AAAFrontCover171026.pdf',
 'http://www.eftaylor.com/exploringblackholes/AAAFrontCover171026.pdf',
 'http://www.eftaylor.com/exploringblackholes/?path=&download=AAFrontMatter170421v6.pdf',
 'http://www.eftaylor.com/exploringblackholes/AAFrontMatter170421v6.pdf',
 'http://www.eftaylor.com/exploringblackholes/AAFrontMatter170421v6.pdf',
 'http://www.eftaylor.com/exploringblackholes/?path=&download=AContents170511v1.pdf',
 'http://www.eftaylor.com/exploringblackholes/AContents170511v1.pdf',
 'http://www.eftaylor.com/exploringblackholes/AContents170511v1.pdf',
 'http://www.eftaylor.com/exploringblackholes/?path=&download=Ch01Speeding1

# hit [page](http://localhost:9999) from local flask server


In [10]:
dta('http://localhost:9999', ['mp3', 'pdf'])

URLError: <urlopen error [Errno 61] Connection refused>

# Fetch links from flask, and save in local filesystem

In [11]:
def dtaFetch(dir, url, types=['pdf']):
    "copy dta urls into dir"
    with withChdir(dir):
        # relative paths are easier
        # save the top level url 
        with open('base-url.txt', "w") as fd:
            fd.write(url + '\n')
        urls = dta(url, types)

        for url in urls:
            # make a file name from the url
            tup = urllib.parse.urlparse(url)
            path = tup.path
            name = os.path.split(path)[1]
            url = urllib.parse.quote(url, safe='/:')

            print('save', url, name)
            try:                
                with urllib.request.urlopen(url) as net:
                    # grab it in one chunk
                    s = net.read()
                with open(name, 'bw') as fd:
                    # binary write to file system
                    fd.write(s)
            except Exception as e:
                print('missing url: ', url, e)

In [12]:
dtaFetch('dta-download', 'http://localhost:9999', ['mp3', 'pdf'])

FileNotFoundError: [Errno 2] No such file or directory: 'dta-download'

In [13]:
'http://www.eftaylor.com/exploringblackholes/

SyntaxError: EOL while scanning string literal (<ipython-input-13-7ade57c20a18>, line 1)